# Stock NeurIPS2018 Part 1. Data
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*.

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [2]:
## install finrl library
!pip install git+https://github.com/andwir/FinRL-Mods.git

  Cloning https://github.com/andwir/FinRL-Mods.git to /tmp/pip-req-build-0sbrr5lt
  Running command git clone --filter=blob:none --quiet https://github.com/andwir/FinRL-Mods.git /tmp/pip-req-build-0sbrr5lt
  Resolved https://github.com/andwir/FinRL-Mods.git to commit 25f17a345dda33beb9bab0384ef89554d80fa4b3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-lde43xlv/elegantrl_0e4618024479477ea0aacafef95004e4
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-lde43xlv/elegantrl_0e4618024479477ea0aacafef95004e4
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 8ea76afc3e7f1564ae9f0e69e70254116d575fe9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 3.8 MB/s eta 0:00:00
 

In [4]:
!pip install pandas_market_calendars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 8.9 MB/s eta 0:00:00


In [5]:
import pandas as pd
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS
from finrl.config import *
import itertools

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

In [11]:
TRAIN_START_DATE = '2025-01-01'
TRADE_END_DATE = '2025-01-31'
aapl_df_yf = yf.download(tickers = "aapl", start=TRAIN_START_DATE, end=TRADE_END_DATE)

[*********************100%***********************]  1 of 1 completed


In [12]:
aapl_df_yf.head()

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2025-01-02,243.263199,248.500565,241.238085,248.330961,55740700
2025-01-03,242.774368,243.592387,241.307905,242.774368,40244100
2025-01-06,244.410416,246.734810,242.614744,243.722074,45045600
2025-01-07,241.627136,244.959095,240.769205,242.395272,40856000
2025-01-08,242.115952,243.123531,239.472335,241.337830,37628900


### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [13]:
TRAIN_END_DATE = '2025-04-30'

In [14]:
aapl_df_finrl = YahooDownloader(start_date = TRAIN_START_DATE,
                                end_date = TRAIN_END_DATE,
                                ticker_list = ['aapl']).fetch_data()

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (80, 8)


In [15]:
aapl_df_finrl.head()

Price,date,close,high,low,open,volume,tic,day
0,2025-01-02,243.263199,248.500565,241.238085,248.330961,55740700,aapl,3
1,2025-01-03,242.774368,243.592387,241.307905,242.774368,40244100,aapl,4
2,2025-01-06,244.410416,246.734810,242.614744,243.722074,45045600,aapl,0
3,2025-01-07,241.627136,244.959095,240.769205,242.395272,40856000,aapl,1
4,2025-01-08,242.115952,243.123531,239.472335,241.337830,37628900,aapl,2


## Data for the chosen tickers

In [31]:
#AW config_tickers.DOW_30_TICKER
config_tickers.DOW_30_TICKER
#config_tickers.DOW_30_TICKER.append('SRB.L')
#config_tickers.DOW_30_TICKER.remove('SRB.L')


['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [32]:
'''
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2021-10-29'
'''
TRAIN_START_DATE = '2021-01-01'
TRAIN_END_DATE = '2024-02-01'
TRADE_START_DATE = '2024-02-01'
TRADE_END_DATE = '2025-05-21'

In [33]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (33000, 8)


In [34]:
df_raw.head()

Price,date,close,high,low,open,volume,tic,day
0,2021-01-04,126.239685,130.336789,123.654604,130.248998,143301900,AAPL,0
1,2021-01-04,196.605713,200.587092,194.012171,200.587092,3088200,AMGN,0
2,2021-01-04,111.530670,115.083327,110.406291,114.610899,3472100,AXP,0
3,2021-01-04,202.720001,210.199997,202.490005,210.000000,21225600,BA,0
4,2021-01-04,166.714310,170.219751,164.975324,167.492285,4078300,CAT,0


# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [35]:
#AW doesn't handle missin gdata
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (1099, 8)
Successfully added vix


Successfully added turbulence index


In [36]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [22]:
processed_full.head()

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2021-01-04,AAPL,126.239685,130.336789,123.654604,130.248998,143301900.0,0.0,0.0,129.227402,124.812775,100.0,66.666667,100.0,126.239685,126.239685,26.969999,0.0
1,2021-01-04,AMGN,196.605713,200.587092,194.012171,200.587092,3088200.0,0.0,0.0,129.227402,124.812775,100.0,66.666667,100.0,196.605713,196.605713,26.969999,0.0
2,2021-01-04,AXP,111.530670,115.083327,110.406291,114.610899,3472100.0,0.0,0.0,129.227402,124.812775,100.0,66.666667,100.0,111.530670,111.530670,26.969999,0.0
3,2021-01-04,BA,202.720001,210.199997,202.490005,210.000000,21225600.0,0.0,0.0,129.227402,124.812775,100.0,66.666667,100.0,202.720001,202.720001,26.969999,0.0
4,2021-01-04,CAT,166.714310,170.219751,164.975324,167.492285,4078300.0,0.0,0.0,129.227402,124.812775,100.0,66.666667,100.0,166.714310,166.714310,26.969999,0.0


# Part 4: Save the Data

### Split the data for training and trading

In [23]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

23220
9750


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [24]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')